# Neural Network

## Implementasi Oleh Kelompok

In [3]:
import math
import numpy as np

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [4]:
import random

class NeuralNet:
    
    # Constructor
    def __init__(self, nodes_n_in_hidden_layers, learning_rate, momentum):
        nodes_n_in_hidden_layers.append(1) # satu node buat output
        self.nodes_n_in_hidden_layers = nodes_n_in_hidden_layers
        self.inputs = []
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.outputs = [] # output dari setiap node pada satu iterasi
        self.weights = [] # weight dari setiap edge pada satu iterasi
        self.biases = [] # bias dari setiap node pada satu iterasi
        self.weight_biases = []
        self.local_gradients = [] # local gradient dari setiap node pada satu iterasi
        self.delta_weights = [] # delta weight dari setiap edge pada satu iterasi
        self.delta_biases = [] # delta bias dari setiap node pada satu iterasi
        self.layer_nodes = [] # node-node pada layer-layer
        self.v = [] # v pada setiap node
        self.targets = []
    
    # Feed Forward
    def feed_forward(self, datum_idx):
        for i in range (1, len(self.layer_nodes)):
            for j in range (0, len(self.layer_nodes[i])):
                current_node = self.layer_nodes[i][j]
                weights = []
                weights.append(self.biases[current_node])
                inputs = []
                inputs.append(1)
                for k in range(0, len(self.layer_nodes[i-1])):
                    if (self.weights[self.layer_nodes[i-1][k]][current_node] != None):
                            weights.append(self.weights[self.layer_nodes[i-1][k]][current_node])
                    
                    if (i==1):
                        inputs.append(self.inputs[datum_idx][self.layer_nodes[i-1][k]])
                    else:
                        inputs.append(self.outputs[self.layer_nodes[i-1][k]])
                v = np.dot(inputs, weights)
                self.v[current_node] = v
                self.outputs[current_node] = sigmoid(v)
                
    # Back Propagation
    def back_propagation(self, datum_idx):
        for i in range(len(self.layer_nodes)-1, 0, -1):
            for j in range(len(self.layer_nodes[i])-1, -1, -1):
                current_node = self.layer_nodes[i][j]
                if (i == len(self.layer_nodes)-1):
                    v = self.v[current_node]
                    sig_v = sigmoid(v)
                    self.local_gradients[current_node] = self.local_gradients[current_node] + (sig_v * (1 - sig_v) * (self.targets[datum_idx] - self.outputs[current_node]))
                else:
                    v = self.v[current_node]
                    sig_v = sigmoid(v)
                    weight_delta = 1
                    for k in range(0, len(self.layer_nodes[i+1])):
                        weight_delta = weight_delta * self.local_gradients[self.layer_nodes[i+1][k]] * self.weights[current_node][self.layer_nodes[i+1][k]]
                    self.local_gradients[current_node] = self.local_gradients[current_node] + (sig_v * (1 - sig_v) * weight_delta)            
                
    # Update Weight
    def update_weight(self):
        for i in range(0, len(self.layer_nodes)-1):
            for j in range(0, len(self.layer_nodes[i])):
                current_node = self.layer_nodes[i][j]
                for k in range(0, len(self.layer_nodes[i+1])):
                    current_next_node = self.layer_nodes[i+1][k]
                    self.delta_weights[current_node][current_next_node] = self.momentum * self.delta_weights[current_node][current_next_node] + self.learning_rate * self.local_gradients[current_next_node] * self.outputs[current_next_node]
                    self.weights[current_node][current_next_node] += self.delta_weights[current_node][current_next_node]
        for i in range(1, len(self.biases)):
            new_bias = self.biases[i] + self.momentum  * self.delta_biases[i] + self.learning_rate * self.local_gradients[i]
            self.delta_biases[i] = new_bias - self.biases[i]
            self.biases[i] = new_bias
            
    # Fit
    def fit(self, X, Y, batch_size, max_iter, threshold): # data = array of arrays
        #data[0] ke n merupakan label
        #nodes_n_in_hidden_layers[0] merupakan jumlah input
        self.nodes_n_in_hidden_layers.insert(0, len(X[0])) 
        
#         inputs = X
#         for i in range (0, len(inputs)):X
#             self.targets.append(inputs[i].pop(len(inputs[i])-1))
        self.targets = Y
        self.inputs = X
        
        n_nodes = 0
        init_weight = 1 # Weights diinisalisasi 0
        
        # Inisialisasi output, bias, local gradient, v, dan delta bias di setiap node pada layer
        for i in range(0, len(self.nodes_n_in_hidden_layers)):
            l_nodes = []
            for j in range(0, self.nodes_n_in_hidden_layers[i]):
                self.outputs.append(0)
                self.v.append(0)
                self.biases.append(0) #asumsi x bias = 1
                self.local_gradients.append(0)
                self.delta_biases.append(0)
                l_nodes.append(n_nodes)
                n_nodes += 1
            self.layer_nodes.append(l_nodes)
        
        for i in range(0, n_nodes):
            self.weights.append([])
            self.delta_weights.append([])
            for j in range(0, n_nodes):
                self.weights[i].append(None)
                self.delta_weights[i].append(None)
            
        current_node = 0
        for i in range(0, len(self.nodes_n_in_hidden_layers)-1):
            if (i < len(self.nodes_n_in_hidden_layers)-1):
                next_layer_first_node = current_node + self.nodes_n_in_hidden_layers[i]
                for j in range(0, self.nodes_n_in_hidden_layers[i]):
                    for k in range(0, self.nodes_n_in_hidden_layers[i+1]):
                        self.weights[current_node][k+next_layer_first_node] = random.uniform(-0.5,0.5)
                        self.delta_weights[current_node][k+next_layer_first_node] = 0
                    current_node += 1
        
        n_batch = math.ceil(len(Y)/batch_size)
        
        n_iter = 0
        error = 100
        while (n_iter < max_iter and error > threshold):
            datum_idx = 0
            for i in range(0, n_batch):
                # Mengembalikan local_gradient menjadi 0
                current_node = 0
                for i in range(0, len(self.nodes_n_in_hidden_layers)-1):
                    if (i < len(self.nodes_n_in_hidden_layers)-1):
                        next_layer_first_node = current_node + self.nodes_n_in_hidden_layers[i]
                        for j in range(0, self.nodes_n_in_hidden_layers[i]):
                            for k in range(0, self.nodes_n_in_hidden_layers[i+1]):
                                self.local_gradients[current_node] = 0
                            current_node += 1
                j = 0
                accum_error = 0
                num_datum = 0
                while (j < batch_size):
                    if (datum_idx < len(Y)):
                        self.feed_forward(datum_idx)
                        accum_error = accum_error + (0.5 * ((self.targets[datum_idx] - self.outputs[-1])**2))
                        num_datum = num_datum + 1
                        self.back_propagation(datum_idx)
                        datum_idx += 1
                        j += 1
                    else:
                        j = batch_size + 1
                error = accum_error / num_datum
                if (error < threshold):
                    break
                self.update_weight()
            
            n_iter += 1
        
    # Predict
    def predict(self, data_test):
        test_outputs = []
        feed_forward_result = []
        self.inputs = data_test
        
        for i in range (0, len(data_test)):
            self.feed_forward(i)
            test_outputs.append(self.outputs[-1])
        
        return test_outputs
    
    def predict_label(test_outputs):
        test_label = []
        for i in range (0, len(test_outputs)):
            if test_outputs[i] >= 0.5:
                test_label.append(1)
            else:
                test_label.append(0)
            
                

## Implementasi dengan Keras

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

class KerasNeuralNet:
    
    # Construction
    def __init__(self, nodes_n_in_hidden_layers, learning_rate, momentum):
        nodes_n_in_hidden_layers.append(1) # satu node buat output
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.model = Sequential()
        self.model.add(Dense(output_dim=nodes_n_in_hidden_layers[0], input_dim=4, activation="sigmoid"))
        for i in range(1, len(nodes_n_in_hidden_layers)):
            self.model.add(Dense(output_dim=nodes_n_in_hidden_layers[i], input_dim=nodes_n_in_hidden_layers[i-1], activation='sigmoid'))
        sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=True)
        self.model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
        
    def fit(self, X, Y, batch_size):
        self.model.fit(X, Y, batch_size=batch_size)
    
    def predict(self, X):
        return self.model.predict(X)

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

## Membaca Data Tennis

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Load data
dataframe = pd.read_csv('tennis.csv')

# Transform outlook, temperature, humidity, and windy to numerical values
le = preprocessing.LabelEncoder()
encoded = dataframe.apply(le.fit_transform)
dataset = encoded.values

# X and Y values
X = dataset[:,0:4]
Y = dataset[:,4]

# Rescale min and max for X
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


## Fitting dengan Hasil Implementasi Kelompok

In [13]:
nn = NeuralNet([6], 0.25, 0.0001)
nn.fit(rescaledX, Y, 1, 5, 0.01)
nn.predict(rescaledX)
# nn.predict(data_test)

[0.8755414339774144,
 0.8759338659938654,
 0.8753646726586333,
 0.8786677680326138,
 0.871920238679045,
 0.8724222697332084,
 0.8726554308598974,
 0.8782679104783107,
 0.8720127556101648,
 0.8776104941294605,
 0.8763023545747011,
 0.8795938093607254,
 0.875118303569057,
 0.8791866494282785]

## Fitting dengan Keras

In [1]:
knn = KerasNeuralNet([6], 0.25, 0.0001)

# Train model
knn.fit(rescaledX, Y, 1)

# Predict
knn.predict(rescaledX)

NameError: name 'KerasNeuralNet' is not defined